#### Forecasting: Imports and Setup

In [ ]:
%reload_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
from keras.callbacks  import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Lambda, Dropout, SimpleRNN, Dense, LSTM, RepeatVector, Input, TimeDistributed, concatenate
from keras import regularizers

import IPython, IPython.display, os, datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ts_utils
import ts_plot_utils

mpl.rcParams['figure.figsize'] = (14, 4)
mpl.rcParams['axes.grid'] = True

In [ ]:
# Read the data
df, df_scaled_trn, df_scaled_tst, scaler = ts_utils.load_file()
print("Training Data:")
display(df_scaled_trn)

input_slice  = slice(0, len(df_scaled_trn.columns) )
label_slice  = slice(0,3)
window_len   = 24
ouput_len    = 1
batch_size   = 64

inp_feat_len    = input_slice.stop - (input_slice.start or 0)
ouput_feat_len  = label_slice.stop - (label_slice.start or 0)

ds_trn        = tf.data.Dataset.from_tensor_slices(df_scaled_trn[df_scaled_trn.columns[input_slice]])
ds_tst        = tf.data.Dataset.from_tensor_slices(df_scaled_tst[df_scaled_trn.columns[input_slice]])
window_trn    = ts_utils.window(ds_trn, window_len, ouput_len, label_slice, batch_size=batch_size,)
window_tst    = ts_utils.window(ds_tst, window_len, ouput_len, label_slice, batch_size=batch_size )
window_trn100 = ts_utils.window(ds_trn, window_len, ouput_len, label_slice, batch_size=100000  )
window_tst100 = ts_utils.window(ds_tst, window_len, ouput_len, label_slice, batch_size=100000  )


In [ ]:
%%script echo Skipping - Testing Window -comment this line to test

# Check if your window logic is correctly working
# Note it will print only the first item in the batch
#
i =0
for w in window_trn.take(i+1):
    break;

print(f"Sample a window at index {i} verify the window is working:\n")
print(f"{w[0].numpy().shape}\n{w[0][0].numpy()}\n=>:{w[1].numpy().shape}\n{w[1][0].numpy()} \n")


In [ ]:
%%script echo Skipping - Inverse transform comment this line to test

display(df_scaled_trn[:4])
display(df[df.columns[1:]][:4])
pd.DataFrame(scaler.inverse_transform(df_scaled_trn[:4].values), columns=df_scaled_trn.columns)

In [ ]:
def compile_fit(model, window_trn, window_tst= None, opt=None, patience=3, epochs=1, **kwargs):
    early_stop = EarlyStopping(monitor='val_loss', patience=patience, mode='min', restore_best_weights = True)

    loss = tf.keras.losses.MeanSquaredError()
    opt  = opt or tf.keras.optimizers.Adam()
    mets = [tf.keras.metrics.MeanAbsoluteError()]

    ##=> Other options you can try
    #learning_rate = 1e-6
    #opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    #opt = tf.keras.optimizers.SGD()
    #loss=tf.keras.losses.Huber()

    model.compile(loss= loss, optimizer= opt, metrics=mets)

    history = []
    if (window_trn is not None):
        history = model.fit(window_trn, epochs=epochs, validation_data=window_tst, 
                                workers=4, use_multiprocessing=True, callbacks=[early_stop], **kwargs)

    return history

# This commonLayer, a layer that is common to all models

def getCommonLayer(ouput_len, ouput_feat_len):
    op_len = ouput_len * ouput_feat_len;
    commonLayer = [
        # Shape => [batch, 1, out_len * #features]
        tf.keras.layers.Dense( op_len, kernel_initializer=tf.initializers.zeros()),
        
        # Shape => [batch, out_steps, features]
        tf.keras.layers.Reshape([ouput_len, ouput_feat_len])
    ]
    return commonLayer

performance = {}
models = []

In [ ]:
# LSTM Model
lstm_model = tf.keras.Sequential([
        # Shape [batch, time, features] => [batch, lstm_units].
        # Adding more `lstm_units` just overfits more quickly.

        tf.keras.layers.LSTM(32, return_sequences=False)
    ]+ getCommonLayer(ouput_len, ouput_feat_len),
    name = "LSTM"
)
models.append(lstm_model)


In [ ]:
# Linear Model
linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :])
    ] + getCommonLayer(ouput_len, ouput_feat_len),
    name = "Linear"
)
models.append(linear_model)

In [ ]:
#Simple RNN
srnn_model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(40)]
    + getCommonLayer(ouput_len, ouput_feat_len)
    , name="SimpleRNN")

models.append(srnn_model)

# Add a Dense Layer
dlinear_model = tf.keras.Sequential([
        # Take the last time-step.
        # Shape [batch, time, features] => [batch, 1, features]
        tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
        tf.keras.layers.Dense(512, activation='relu')
    ] +  getCommonLayer(ouput_len, ouput_feat_len),
    name = "DenseLinear"
)
models.append(dlinear_model)


# CNN
CONV_WIDTH = 3
conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    ] + getCommonLayer(ouput_len, ouput_feat_len),
    name = "Conv"
)
models.append(conv_model)

In [ ]:
dim = 256

# Create Autoencoder Layer
input_layer = Input(shape=(window_len, inp_feat_len), dtype='float32', name='input')
memory_layer = LSTM(dim, return_sequences=True)(input_layer)
memory_layer = LSTM (dim//2, return_sequences=False)(memory_layer)
repeated_lyr = RepeatVector(window_len)(memory_layer)
memory_layer = LSTM (dim//2, return_sequences=True)(repeated_lyr)
memory_layer = LSTM (dim,  return_sequences=True)(memory_layer)
decoded_inputs = TimeDistributed(Dense(units=inp_feat_len, activation='linear'))( memory_layer)

dropout_input = Dropout(0.2)(input_layer)
concat_layer = concatenate([dropout_input, decoded_inputs])
memory_layer = LSTM(units=dim, 
                    kernel_regularizer = regularizers.l1_l2(l1= 0, l2= 0), 
                    recurrent_regularizer = regularizers.l1_l2(l1= 0, l2= 0), 
                    return_sequences=False)(concat_layer)

# => Note this is same as getCommonLayer(ouput_len, ouput_feat_len, memory_layer)
#
# preds = Dense(units=ouput_feat_len*ouput_len)(memory_layer)
preds = Dense(units=ouput_feat_len*ouput_len, activation='linear')(memory_layer)
preds = tf.keras.layers.Reshape([ouput_len, ouput_feat_len])(preds)
umodel = Model(input_layer, preds, name="Uber")

#models.append(umodel)

# Compile and Evaluate All the models

In [ ]:
for i, model in enumerate(models):
    print(f"Now Compiling {i+1}/{len(models)} {model.name} ")
    #history = compile_fit(model, window_trn, window_tst, epochs=5, verbose=1)
    history = compile_fit(model, None, window_tst, epochs=5, verbose=1)
    IPython.display.clear_output()

# Plot graphs
#performance={}
performance = ts_plot_utils.plot_performance(models, window_trn100, window_tst100, performance=performance, reeval=1)

In [ ]:
performance

### Predictions

In [ ]:
model = umodel
ydf, pdf = ts_plot_utils.predict_and_plot( model, window_trn100, window_tst100, howmany=1024* 1024,
                        plot_start=-200, df=None, scaler=None, label_slice=None)
;

In [ ]:
model = linear_model
ydf, pdf = ts_plot_utils.predict_and_plot( model, window_trn100, window_tst100, howmany=1024* 1024,
                        plot_start=-200, df=None, scaler=None, label_slice=None)
;

### The END